In [ ]:
!gdown "1xMDTrHIK_gw3vLsmLZc8N2vCXNGouWQS&confirm=t"
!unzip seminar_objdet_retina_oi5_ball.zip -d seminar_objdet_retina_oi5_ball

In [58]:
!git clone -b autumn_2023 https://github.com/AlekseySpasenov/dl-course.git

Cloning into 'dl-course'...
remote: Enumerating objects: 447, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 447 (delta 16), reused 26 (delta 9), pack-reused 394
Receiving objects: 100% (447/447), 171.92 MiB | 36.93 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [2]:
import os
import sys
import cv2
import json
import matplotlib.pyplot as plt

import numpy as np
import torch
from torch import nn
from torchvision.transforms.functional import to_tensor, normalize

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device is:", device)


Device is: cuda


In [61]:
# rename dir
from dl_course.lecture6.visualization import show_image, draw_predictions

In [62]:
def make_tensor(image, device=None):
    tensor = to_tensor(image).unsqueeze(0)
    tensor = normalize(tensor,
                       mean=(0.485, 0.456, 0.406),
                       std=(0.229, 0.224, 0.225))
    if device:
        tensor = tensor.to(device)
    return tensor

In [63]:
def run_and_show(model, image_or_tensor, threshold, class_to_label_map, verbose=False):
    if isinstance(image_or_tensor, np.ndarray):
        tensor = make_tensor(image_or_tensor)
        image = image_or_tensor
    elif isinstance(image_or_tensor, torch.Tensor):
        image = (image_or_tensor.numpy()[0] * 0.25) + 0.5
        tensor = image_or_tensor.permute(0, 3, 1, 2)
    else:
        raise NotImplementedError(type(image_or_tensor))

    with torch.no_grad():
        nms_scores, nms_classes, bboxes = model(tensor)

    nms_scores = nms_scores.cpu()
    nms_classes = nms_classes.cpu()
    bboxes = bboxes.cpu()

    if verbose:
        print(nms_scores.size(), nms_scores[:8])
        print(nms_classes.size(), nms_classes[:8])
        print(bboxes.size(), bboxes[:8])

    image_with_predictions = draw_predictions(image, bboxes, nms_scores, nms_classes, class_to_label_map, threshold=threshold)
    show_image(image_with_predictions)

In [64]:
with open("./seminar_objdet_retina_oi5_ball/coco_id_to_name.json", "rt") as fp:
    coco_class_to_label_map = json.load(fp)
coco_class_to_label_map = {int(k) - 1: v for k, v in coco_class_to_label_map.items()}

In [65]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask,
        hidden_layer,
        num_classes,
    )

    return model

In [66]:
from torchvision.transforms import v2 as T


def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = get_model_instance_segmentation(2)
model.to(device)

In [72]:
%reload_ext autoreload

import tqdm

from torchvision import transforms
from torch.utils.data import DataLoader

from dl_course.lecture6.dataset import DetectionDataset
from dl_course.lecture6.retinanet.dataloader import Normalizer, Resizer, Augmenter, collater, AspectRatioBasedSampler, UnNormalizer

In [73]:
train_transforms = transforms.Compose([
    Normalizer(), Augmenter(), Resizer()
])

val_transforms = transforms.Compose([
    Normalizer(), Resizer()
])

In [75]:
train_dataset = DetectionDataset(data_dict_file="./seminar_objdet_retina_oi5_ball/oi5_ball_filename_to_bbox_train.json",
                                 transforms=train_transforms, add_path = './seminar_objdet_retina_oi5_ball/')
train_sampler = AspectRatioBasedSampler(train_dataset, batch_size=4, drop_last=False) # deepdive
train_dataloader = DataLoader(train_dataset, num_workers=8, collate_fn=collater, batch_sampler=train_sampler)

val_dataset = DetectionDataset(data_dict_file="./seminar_objdet_retina_oi5_ball/oi5_ball_filename_to_bbox_val.json",
                               transforms=val_transforms, add_path = './seminar_objdet_retina_oi5_ball/')
val_sampler = AspectRatioBasedSampler(val_dataset, batch_size=4, drop_last=False)
val_dataloader = DataLoader(val_dataset, num_workers=8, collate_fn=collater, batch_sampler=val_sampler)

In [107]:
%%shell
rm -rf vision
rm utils.py transforms.py coco_eval.py engine.py coco_utils.py
# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout release/2.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 415301, done.
remote: Counting objects: 100% (24464/24464), done.
remote: Compressing objects: 100% (1926/1926), done.
remote: Total 415301 (delta 23608), reused 22888 (delta 22526), pack-reused 390837
Receiving objects: 100% (415301/415301), 797.86 MiB | 36.19 MiB/s, done.
Resolving deltas: 100% (385830/385830), done.
Branch 'release/2.0' set up to track remote branch 'release/2.0' from 'origin'.
Switched to a new branch 'release/2.0'


In [94]:
#from torch._six import inf
from torch import inf

In [108]:
from engine import train_one_epoch, evaluate

In [120]:
import numpy as np
import gc

In [121]:
def routine(model, scheduler, train_loader, val_loader):
    model.train()
    epoch_loss = []
    torch.cuda.empty_cache()
    for iter_num, data in tqdm.tqdm(enumerate(train_dataloader), total=len(train_dataloader)):

        optimizer.zero_grad()

        loss = model(data['img'].to(device).float(), data['annot'].to(device))

        if bool(loss == 0):
            continue

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

        epoch_loss.append(float(loss.item()))
        del data

    gc.collect()
    torch.cuda.empty_cache()

    print('Epoch: {} | Iteration: {} | Epoch loss: {:1.5f}'\
          .format(epoch_num, iter_num, np.mean(epoch_loss)))
    scheduler.step(np.mean(epoch_loss)) # should be val loss

In [122]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)

In [123]:
epochs = 5
for epoch_num in range(epochs):
    routine(model, scheduler, train_dataloader, val_dataloader)

  0%|          | 0/719 [00:06<?, ?it/s]


IndexError: ignored

Я не знаю как адаптировать код с семинара, чтоб торчовая модель принимала дата лоудер для ретины

Я больше не могу, я прочитал много кода, я всё равно не могу это сделать 🚑🚑🚑🚑🚑🚑🚑🚑🚑🚑🚑🚑🚑🚑🚑🚑